In [98]:
!pip install keras

You should consider upgrading via the '/Users/taliyas/PycharmProjects/NER/venv/bin/python -m pip install --upgrade pip' command.


In [99]:
!pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/taliyas/PycharmProjects/NER/venv/lib/python3.9/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: tensorflow-text


In [100]:
!pip install pandas openpyxl

You should consider upgrading via the '/Users/taliyas/PycharmProjects/NER/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install transformers

### IMPORT

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_text
import tensorflow_hub as hub
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.python.keras import metrics
from tensorflow.keras import metrics

### Take the labeled data that we have already prepared with the help of queries to ChatGPT

In [1]:


# Load the CSV file
data = pd.read_csv('GPTClassification.csv')
data.shape

(364, 2)

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  364 non-null    object
 1   category  362 non-null    object
dtypes: object(2)
memory usage: 5.8+ KB


In [3]:
data.head(10)

,Sentence,category
0,"""I think there should be many various sitting ...",Land use
1,"""It should be distinctive but still matching t...",Local identity
2,"""It should be clear where the entrance is and ...",Land use
3,"""well I think that the spaces should be focuse...","Land use, Future of work"
4,"""well it could be achieved through different m...",Land use
5,"""yes, the current building, as unsuitable as i...",Local identity
6,"""maybe, however Im interested in your suggesti...",other
7,"""i find that a very correct approach... how di...",other
8,"""correct again """,other
9,"""well maybe it is a possibility to experiment ...",Local identity


In [4]:
empty_string_count = (data['category'] == 'other').sum()
print("There is ", empty_string_count, " sentence that are not labeled")

There is  93  sentence that are not labeled


In [5]:
# delete the NuN values
data = data.dropna()

In [6]:
total_duplicate_titles = sum(data["Sentence"].duplicated())
print(f"There are {total_duplicate_titles} duplicate Sentences.")

There are 14 duplicate Sentences.


In [7]:
# Create a boolean mask for duplicated sentences
mask = data["Sentence"].duplicated(keep=False)

# Print the rows with duplicated sentences
data[mask]


,Sentence,category
17,"""no""",other
51,"""What spaces should be in the building?""","Future of work, Land use"
54,"""What should the building look like? """,Local identity
71,"""What spaces should be in the building?""",Land use
74,"""What should the building look like? """,Local identity
85,"""Hi Zaha""",other
97,"""What would you like to know""",other
123,"""spaces""","Environment and climate resilience, Land use"
127,"""trees grass""","Environment and climate resilience, Land use"
129,"""spaces""","Environment and climate resilience, Land use"


In [8]:
data = data[~data["Sentence"].duplicated()]
print(f"There are {len(data)} rows in the deduplicated dataset.")

# There are some categories with occurrence as low as 1.
print("categories upon once only:",sum(data["category"].value_counts() == 1))

# How many unique terms?
print("unique categories:",data["category"].nunique())

There are 348 rows in the deduplicated dataset.
categories upon once only: 14
unique categories: 31


We need to handle the categories upon only once, its problematic for multi-label text classifier

In [9]:
def get_single_occurrence_categories(data):
    # Get the categories that appear only once
    single_occurrence_categories = data['category'].value_counts()
    single_occurrence_categories = single_occurrence_categories[single_occurrence_categories == 1].index

    # Get the rows where 'category' is in the list of single-occurrence categories
    once_sen = data[data['category'].isin(single_occurrence_categories)]
    return once_sen


In [10]:
once_sen = get_single_occurrence_categories(data)
once_sen


,Sentence,category
15,"""hi, i think if there´s more free use co-worki...","Local identity, Environment and climate resili..."
19,"""The building should be more open for the soci...","Local identity, Environment and climate resili..."
22,"""To have a cafe on the ground floor would be n...","Local identity, Mobility (transport)"
53,"""Can we have a garden?""","Land use, Environment and climate resilience"
110,"""I agree! Architecture students should be insp...","Environment and climate resilience, Future of ..."
145,"""The building should have several spaces for g...","Environment and climate resilience, Local iden..."
183,"""Can I 3d print infra lightweight concrete?""",Future of work; Land use
194,"""I think it should be more inviting compared t...",Local identity; Land use
198,"""There should be greenery aroud the building, ...",Environment and climate resilience; Land use
226,"""I think that a good architecture school shoul...",Land use; Local identity


In [11]:
# Replace ";" with ","
data['category'] = data['category'].str.replace(';', ',')


In [12]:
data["category"] = data["category"].apply(
    lambda x: x.split(', ')
)
data["category"].values[:5]


array([list(['Land use']), list(['Local identity']), list(['Land use']),
       list(['Land use', 'Future of work']), list(['Land use'])],
      dtype=object)

In [13]:
once_sen = get_single_occurrence_categories(data)
once_sen

,Sentence,category
15,"""hi, i think if there´s more free use co-worki...","[Local identity, Environment and climate resil..."
19,"""The building should be more open for the soci...","[Local identity, Environment and climate resil..."
22,"""To have a cafe on the ground floor would be n...","[Local identity, Mobility (transport)]"
53,"""Can we have a garden?""","[Land use, Environment and climate resilience]"
110,"""I agree! Architecture students should be insp...","[Environment and climate resilience, Future of..."
145,"""The building should have several spaces for g...","[Environment and climate resilience, Local ide..."
239,"""Hello Zaha, nice to meet you! I'm glad to be ...","[Environment and climate resilience, Local ide..."
316,"""I hope in office room or big classroom there ...",[Indoor comfort]
335,"""Who's your favorite architect what's your fav...","[Mobility (transport), Local identity]"


The categories that appear once in the data we took from 14 to 9.
Now for those 9 - we will separate them to different rows

In [14]:
# Create a list to store the new rows
new_rows = []

# Iterate over the rows in the dataframe where category is in single_occurrence_categories
for idx, row in once_sen.iterrows():
    # Iterate over the categories in the 'category' column
    for cat in row['category']:
        new_row = {'Sentence': row['Sentence'], 'category': [cat]}
        # Only add the new row if it is not already in new_rows
        if new_row not in new_rows:
            new_rows.append(new_row)
    data = data.drop(idx)

# Create a new DataFrame from the new rows
new_rows_df = pd.DataFrame(new_rows)

# Append the new rows to the dataframe using concat
data = pd.concat([data, new_rows_df], ignore_index=True)


In [15]:
# Convert lists in 'category' to tuples
new_rows_df['category'] = new_rows_df['category'].apply(tuple)

# Then check for duplicates
total_duplicate_titles = new_rows_df.duplicated().sum()
print(f"There are {total_duplicate_titles} duplicates.")



There are 0 duplicates.


In [16]:
# There are some categories with occurrence as low as 1.
print(once_sen['Sentence'][19])
data[data['Sentence'] == once_sen['Sentence'][19]]
# this cell for checking it was separated

"The building should be more open for the society."


,Sentence,category
342,"""The building should be more open for the soci...",[Local identity]
343,"""The building should be more open for the soci...",[Environment and climate resilience]


In [17]:
data.shape

(361, 2)

In [18]:
# Get the categories that appear more than once
multi_occurrence_categories = data['category'].value_counts()
multi_occurrence_categories = multi_occurrence_categories[multi_occurrence_categories > 1].index

# Filter the data to only include rows where 'category' is in the list of multi-occurrence categories
data = data[data['category'].isin(multi_occurrence_categories)]


In [19]:
data.shape

(360, 2)

In [20]:
test_split = 0.3


# Initial train and test split.
train_df, test_df = train_test_split(
    data,
    test_size=test_split,
    stratify=data["category"].values,
)

# Splitting the test set further into validation
# and new test sets.
val_df = test_df.sample(frac=0.5)
test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 252
Number of rows in validation set: 54
Number of rows in test set: 54


In [21]:

categories = tf.ragged.constant(train_df["category"].values)
print(categories)

<tf.RaggedTensor [[b'Environment and climate resilience'], [b'Local identity'],
 [b'Local identity'], [b'Environment and climate resilience'],
 [b'Land use', b'Local identity'], [b'other'], [b'other'],
 [b'Local identity'], [b'Land use'], [b'Land use'], [b'other'],
 [b'Environment and climate resilience'],
 [b'Environment and climate resilience'], [b'Land use'], [b'other'],
 [b'Future of work', b'Local identity'], [b'Future of work', b'Land use'],
 [b'other'], [b'Land use'], [b'Land use'],
 [b'Environment and climate resilience'], [b'Land use'], [b'other'],
 [b'Future of work'], [b'Environment and climate resilience'],
 [b'Land use', b'Future of work'],
 [b'Environment and climate resilience', b'Land use'],
 [b'Local identity', b'Land use'], [b'other'], [b'Land use'], [b'other'],
 [b'Environment and climate resilience', b'Local identity'],
 [b'Environment and climate resilience'], [b'Local identity'],
 [b'Land use'], [b'Environment and climate resilience'], [b'Land use'],
 [b'Environme

## multi-hot encoding
We will convert the  categorical data into a form that can be used by deep learning models.
because we are dealing with multi-label classification problem.


In [22]:
# Convert tensors to numpy arrays and then to sets of tuples for set operations
train_categories_tensor = tf.ragged.constant(train_df["category"].values)
test_categories_tensor = tf.ragged.constant(test_df["category"].values)

train_categories = set(tuple(x) for x in train_categories_tensor.to_list())
test_categories = set(tuple(x) for x in test_categories_tensor.to_list())

categories_not_in_train = test_categories - train_categories

print(categories_not_in_train)


set()


the result is an empty set (set())- it means there are no categories that exist in the test set but not in the training set

In [23]:
categories = tf.ragged.constant(train_df["category"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot", num_oov_indices=0)
lookup.adapt(categories)
vocab = lookup.get_vocabulary()


def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a tuple of vocab terms."""
    hot_indices = np.argwhere(encoded_labels == 1.0)[..., 0]
    return np.take(vocab, hot_indices)


print("Vocabulary:\n")
print(vocab)

Vocabulary:

['Land use', 'other', 'Environment and climate resilience', 'Local identity', 'Future of work', 'Mobility (transport)']


In [24]:
sample_label = train_df["category"].iloc[0]
print(f"Original label: {sample_label}")

label_binarized = lookup([sample_label])
print(f"Label-binarized representation: {label_binarized}")

Original label: ['Environment and climate resilience']
Label-binarized representation: [[0. 0. 1. 0. 0. 0.]]


In [25]:
train_df["Sentence"].apply(lambda x: len(x.split(" "))).describe()


count    252.000000
mean      13.515873
std       23.430234
min        1.000000
25%        5.000000
50%        8.000000
75%       13.000000
max      179.000000
Name: Sentence, dtype: float64

 the 75th percentile is at 13, meaning that 75% of the sentences have a length of 13 or less.
 and the maximum length observed is 179

In [26]:

max_seqlen = 20
batch_size = 128
padding_token = "<pad>"
auto = tf.data.AUTOTUNE


def make_dataset(dataframe, is_train=True):
    labels = tf.ragged.constant(dataframe["category"].values)
    label_binarized = lookup(labels).numpy()
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["Sentence"].values, label_binarized)
    )
    dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
    return dataset.batch(batch_size)

In [27]:
train_dataset = make_dataset(train_df, is_train=True)
validation_dataset = make_dataset(val_df, is_train=False)
test_dataset = make_dataset(test_df, is_train=False)

In [28]:
test_df

,Sentence,category
62,"""We need more free space """,[Land use]
41,"""minimalistic architecture""",[Local identity]
124,"""open to street""",[Mobility (transport)]
336,"""and should be interesting""",[Local identity]
81,"""What would you design there?""",[Future of work]
109,"""Do you know of the slide in Garching?""",[Local identity]
214,"""Can you tell a good building from a bad one?""",[other]
82,"""Thats not enough""",[other]
220,"""I think that Zaha Hadid design terrible build...",[Local identity]
241,"""the building should optimize the energy consu...",[Environment and climate resilience]


In [29]:

text_batch, label_batch = next(iter(train_dataset))

for i, text in enumerate(text_batch[:5]):
    label = label_batch[i].numpy()[None, ...]
    print(label[0])
    print(f"Abstract: {text}")
    print(f"Label(s): {invert_multi_hot(label[0])}")
    print(" ")


[0. 1. 0. 0. 0. 0.]
Abstract: b'"i see"'
Label(s): ['other']
 
[0. 0. 1. 0. 0. 0.]
Abstract: b'"The building should be made of recyclabe matierals"'
Label(s): ['Environment and climate resilience']
 
[0. 1. 0. 0. 0. 0.]
Abstract: b'"Great Idea!"'
Label(s): ['other']
 
[0. 0. 0. 1. 0. 0.]
Abstract: b'"I want a big playground."'
Label(s): ['Local identity']
 
[1. 0. 0. 0. 0. 0.]
Abstract: b'"What\'s the advantage of using clay on the ceiling?"'
Label(s): ['Land use']
 


In [30]:
vocabulary = set()
train_df["Sentence"].str.lower().str.split().apply(vocabulary.update)
vocabulary_size = len(vocabulary)
print(vocabulary_size)

1010


### BERT MODEL

In [46]:
# Load the LaBSE model and preprocessor from tfhub.dev
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1")
preprocessor = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1")

In [ ]:
def get_labse_embeddings(text, label):
    # Preprocess the text
    encoder_inputs = preprocessor(text)
    # Get the LaBSE embeddings
    sentence_representation = encoder(encoder_inputs)["pooled_output"]
    # Normalize the embeddings
    normalized_sentence_representation = tf.nn.l2_normalize(sentence_representation, axis=-1)
    return (normalized_sentence_representation, label)

# Apply the function to your datasets
train_dataset = train_dataset.map(get_labse_embeddings, num_parallel_calls=auto).prefetch(auto)
validation_dataset = validation_dataset.map(get_labse_embeddings, num_parallel_calls=auto).prefetch(auto)
test_dataset = test_dataset.map(get_labse_embeddings, num_parallel_calls=auto).prefetch(auto)


### TOKENIZER MODEL (TF-IDF)

In [102]:
text_vectorizer = layers.TextVectorization(
    max_tokens=vocabulary_size, ngrams=2, output_mode="tf_idf"
)

# `TextVectorization` layer needs to be adapted as per the vocabulary from our
# training set.
with tf.device("/CPU:0"):
    text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

train_dataset = train_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)
validation_dataset = validation_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)
test_dataset = test_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)

### Create a text classification Model

In [47]:
from tensorflow.keras import layers
def make_model():
    mlp_model = tf.keras.Sequential(
        [
            layers.Dense(512, activation="relu"),
            layers.Dense(256, activation="relu"),
            layers.Dense(128, activation="relu"),
            layers.Dense(64, activation="relu"),
            layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
        ]  # More on why "sigmoid" has been used here in a moment.
    )
    return mlp_model


## Train the model

#### class to evaluate the model

In [49]:
class F1Score(metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = metrics.Precision()
        self.recall = metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

    def result(self):
       precision = self.precision.result()
       recall = self.recall.result()
       return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

In [50]:
epochs = 16

shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "binary_accuracy", #"binary_accuracy" checks for each position if the prediction is correct.
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall'),
        F1Score()
    ]
)

history = shallow_mlp_model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs
)

# Save the model
shallow_mlp_model.save('model_text_clf_bert')

Epoch 1/16
2/2 [==============================] - 112s 29s/step - loss: 0.6803 - binary_accuracy: 0.7163 - precision: 0.2911 - recall: 0.3556 - f1_score: 0.3201 - val_loss: 0.6485 - val_binary_accuracy: 0.8025 - val_precision: 0.5000 - val_recall: 0.0625 - val_f1_score: 0.1111
Epoch 2/16
2/2 [==============================] - 117s 32s/step - loss: 0.6300 - binary_accuracy: 0.8155 - precision: 0.5532 - recall: 0.0915 - f1_score: 0.1571 - val_loss: 0.5791 - val_binary_accuracy: 0.8148 - val_precision: 0.6250 - val_recall: 0.1562 - val_f1_score: 0.2500
Epoch 3/16
2/2 [==============================] - 123s 34s/step - loss: 0.5518 - binary_accuracy: 0.8175 - precision: 0.5513 - recall: 0.1514 - f1_score: 0.2376 - val_loss: 0.5023 - val_binary_accuracy: 0.8117 - val_precision: 0.6364 - val_recall: 0.1094 - val_f1_score: 0.1867
Epoch 4/16
2/2 [==============================] - 116s 31s/step - loss: 0.4790 - binary_accura

INFO:tensorflow:Assets written to: model_text_clf_bert\assets


In [54]:
model_text_clf_bert = tf.keras.models.load_model('model_text_clf_bert', custom_objects={'F1Score': F1Score})

In [ ]:
def plot_evaluate(model):
    def plot_result(item):
        plt.plot(model.history.history[item], label=item)
        plt.plot(model.history.history["val_" + item], label="val_" + item)
        plt.xlabel("Epochs")
        plt.ylabel(item)
        plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
        plt.legend()
        plt.grid()
        plt.show()

    plot_result("loss")
    plot_result("binary_accuracy")
    plot_result("precision")
    plot_result("recall")
    plot_result("f1_score")

### Test part:
1. Predict labels for the test set using the model.
2. Convert these predicted probabilities to binary format (0 or 1) using a threshold of 0.5.
3. For each category:
* Extract the true and predicted labels for this category.
* Calculate and print the accuracy, precision, recall, and F1 score.
* Calculate and plot a confusion matrix for the category.

In [51]:
def pred(model):
    # Generate predictions on the test set
    y_pred = model.predict(test_dataset)

    # Convert predictions to binary format (0 or 1)
    binary_predictions = np.round(y_pred)
    y_true = np.vstack(test_dataset.map(lambda x, y: y).as_numpy_iterator())

    for i in range(binary_predictions.shape[1]):
        print(f"Category {i+1}:\n")

        true_labels = y_true[:, i]
        predicted_labels = binary_predictions[:, i]

        # Calculate metrics
        accuracy = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels)
        recall = recall_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)

        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1}\n")

        # Plot confusion matrix
        cm = confusion_matrix(true_labels, predicted_labels)
        plt.figure(figsize=(5,5))
        sns.heatmap(cm, annot=True, fmt="d")
        plt.title(f'Confusion matrix for category {i+1}')
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.show()


For Summery:
- Category 1 (Land use): Accuracy of 85.19%
- Category 2 (Other): Accuracy of 88.89%
- Category 3 (Environment and climate resilience): Accuracy of 81.48%
- Category 4 (Local identity): Accuracy of 85.19%
- Category 5 (Future of work): Accuracy of 92.59%
- Category 6 (Mobility (transport)): Accuracy of 96.30%

### DISTRUBIOTION
 Lets check for the train and test data how many samples we have from each class:

In [ ]:
from collections import Counter

# Flatten the list of categories
flat_categories = [category for sublist in train_df["category"] for category in sublist]

# Count the occurrences
counts = Counter(flat_categories)

# Print counts in a readable format
for category, count in counts.items():
    print(f" {category}' : {count} .")



In [ ]:
# Flatten the list of categories
flat_categories = [category for sublist in test_df["category"] for category in sublist]

# Count the occurrences
counts = Counter(flat_categories)

# Print counts in a readable format
for category, count in counts.items():
    print(f" {category}' : {count} .")
